## Train Xception Model to recognize Gold Watches

Use Transfer Learning with [Xception Model](https://arxiv.org/abs/1610.02357) in order to predict whether a watch had gold (yellow or rose). Prediction is used to access the formality/casualness of the watch.

Watch is considered "gold", if there is any yellow or rose gold material. When testing, model was confused by gold backgrounds, or odd reflections.

In [1]:
# Utilities
import os

# Analysis
import numpy as np
import pandas as pd

# Keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras import applications
#from keras import backend as K
from keras import callbacks

Using TensorFlow backend.


In [2]:
! ls -R ../../binary_gold_masked/ | wc -l

10546


__Generators__

Generator Inputs

In [3]:
# dimensions of our images.
img_width, img_height = 299, 299

# "flow_from_directory" inputs
batch_size = 8
train_data_dir = 'binary_gold_masked/train/'
validation_data_dir = 'binary_gold_masked/test/'

available_train_files = len(os.listdir(train_data_dir + 'gold/')) \
    + len(os.listdir(train_data_dir + 'not_gold/'))
available_test_files = len(os.listdir(validation_data_dir + 'gold/')) \
    + len(os.listdir(validation_data_dir+'not_gold/'))
    
nb_train_samples  = available_train_files - available_train_files % 8
nb_validation_samples = available_test_files - available_test_files % 8

Make sure that number of files is evenly divisible by batch size (to prevent crashing). If modulo is none zero, remove some files.

In [4]:
available_train_files % 8

0

Create Generators

In [5]:
datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

test_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=True,
    class_mode = 'binary')

Found 7368 images belonging to 2 classes.
Found 3159 images belonging to 2 classes.


__Modeling__

Load Standard Xception Model

In [6]:
model = applications.xception.Xception(include_top=False, weights='imagenet')

Build "Custom" Model

In [7]:
watch_model = Sequential()
watch_model.add(model)
watch_model.add(GlobalAveragePooling2D(name='avg_pool'))
watch_model.add(Dense(2048, activation="relu"))
watch_model.add(Dropout(0.5))
watch_model.add(Dense(1, activation="sigmoid"))

In [8]:
watch_model.layers[0].summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [9]:
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2049      
Total params: 25,059,881
Trainable params: 25,005,353
Non-trainable params: 54,528
_________________________________________________________________


Freeze Xception Layers

In [10]:
for layer in watch_model.layers[0].layers:
    layer.trainable = False

In [11]:
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2049      
Total params: 25,059,881
Trainable params: 4,198,401
Non-trainable params: 20,861,480
_________________________________________________________________


Compile

In [12]:
watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

Run model to set weights on new dense layers

In [14]:
#Fit 

epochs = 5

cb1 = [callbacks.ModelCheckpoint(
    '../models_classification/xception_binary_gold_masked_best1.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb1)

watch_model.save('../models_classification/xception_binary_gold_masked_final1.h5')

Epoch 1/5
921/921 [==============================] - 457s - loss: 0.3120 - binary_accuracy: 0.8643 - val_loss: 0.2611 - val_binary_accuracy: 0.9000
Epoch 2/5
921/921 [==============================] - 456s - loss: 0.3029 - binary_accuracy: 0.8707 - val_loss: 0.2630 - val_binary_accuracy: 0.8918
Epoch 3/5
921/921 [==============================] - 457s - loss: 0.2928 - binary_accuracy: 0.8726 - val_loss: 0.2579 - val_binary_accuracy: 0.8940
Epoch 4/5
921/921 [==============================] - 456s - loss: 0.2954 - binary_accuracy: 0.8743 - val_loss: 0.2580 - val_binary_accuracy: 0.8934
Epoch 5/5
921/921 [==============================] - 457s - loss: 0.2925 - binary_accuracy: 0.8791 - val_loss: 0.2558 - val_binary_accuracy: 0.8892


Unfreeze block 14. 

Last separable convolution first. Ten epochs at a time until testing accuracy "stabilizes" and starts falling below train accuracy. Unfreeze next to last separable convo and repeat.

In [15]:
del watch_model

In [16]:
watch_model = load_model('../models_classification/xception_binary_gold_masked_final1.h5')

In [17]:
#Unfreeze a few more layers and allow to run

# watch_model.layers[0].summary()

# Freeze convolutional layers
for layer in watch_model.layers[0].layers[-3:]:
    layer.trainable = True
    
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2049      
Total params: 25,059,881
Trainable params: 7,362,049
Non-trainable params: 17,697,832
_________________________________________________________________


In [18]:
epochs = 10


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

# watch_model.layers[0].summary()

cb2 = [callbacks.ModelCheckpoint(
    '../models_classification/xception_binary_gold_masked_best2.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb2)


watch_model.save('../models_classification/xception_binary_gold_masked_final2.h5')

Epoch 1/10
921/921 [==============================] - 472s - loss: 0.2820 - binary_accuracy: 0.8793 - val_loss: 0.2504 - val_binary_accuracy: 0.8962
Epoch 2/10
921/921 [==============================] - 472s - loss: 0.2701 - binary_accuracy: 0.8875 - val_loss: 0.2367 - val_binary_accuracy: 0.9124
Epoch 3/10
921/921 [==============================] - 472s - loss: 0.2549 - binary_accuracy: 0.8958 - val_loss: 0.2185 - val_binary_accuracy: 0.9064
Epoch 4/10
921/921 [==============================] - 472s - loss: 0.2480 - binary_accuracy: 0.8954 - val_loss: 0.2076 - val_binary_accuracy: 0.9140
Epoch 5/10
921/921 [==============================] - 472s - loss: 0.2347 - binary_accuracy: 0.9015 - val_loss: 0.2203 - val_binary_accuracy: 0.8978
Epoch 6/10
921/921 [==============================] - 471s - loss: 0.2352 - binary_accuracy: 0.8985 - val_loss: 0.1949 - val_binary_accuracy: 0.9143
Epoch 7/10
921/921 [==============================] - 470s - loss: 0.2394 - binary_accuracy: 0.8992 - val_

In [19]:
watch_model = load_model('../models_classification/xception_binary_gold_masked_final2.h5')

In [8]:
#Unfreeze a few more layers and allow to run

watch_model.layers[0].summary()

# Freeze convolutional layers
for layer in watch_model.layers[0].layers[-6:]:
    layer.trainable = True

epochs = 10


watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy"])

watch_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
epochs = 10

cb3 = [callbacks.ModelCheckpoint(
    '../models_classification/xception_binary_gold_masked_best3.h5',
    monitor='val_loss',
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False, 
    mode='auto', period=1)]

# fine-tune the model
watch_model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = test_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks = cb3)


watch_model.save('../models_classification/xception_binary_gold_masked_final3.h5')

Epoch 1/10
921/921 [==============================] - 481s - loss: 0.1509 - binary_accuracy: 0.9365 - val_loss: 0.1276 - val_binary_accuracy: 0.9432
Epoch 2/10
921/921 [==============================] - 479s - loss: 0.1342 - binary_accuracy: 0.9446 - val_loss: 0.1305 - val_binary_accuracy: 0.9441
Epoch 3/10
921/921 [==============================] - 479s - loss: 0.1330 - binary_accuracy: 0.9411 - val_loss: 0.1268 - val_binary_accuracy: 0.9464
Epoch 4/10
921/921 [==============================] - 479s - loss: 0.1278 - binary_accuracy: 0.9480 - val_loss: 0.1215 - val_binary_accuracy: 0.9489
Epoch 5/10
921/921 [==============================] - 479s - loss: 0.1212 - binary_accuracy: 0.9490 - val_loss: 0.1181 - val_binary_accuracy: 0.9492
Epoch 6/10
441/921 [=============>................] - ETA: 170s - loss: 0.1159 - binary_accuracy: 0.9493

In [ ]:
watch_model = load_model('../models_classification/xception_binary_gold_masked_final3.h5')